In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
dimensoes = [
    'porcentagem_domicilios_sem_coleta_de_lixo',
    'porcentagem_domicilios_sem_ligacao_rede_de_agua',
    'porcentagem_domicilios_sem_banheiros',
    'porcentagem_domicilios_sem_esgoto_ou_fossa',
    # # 'porcentagem_domicilios_tipo_comodo',
    'porcentagem_domicilios_cedidos_outra_forma',
    'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
    'numero_de_banheiros_por_habitante',
    'porcentagem_responsaveis_nao_alfabetizado',
    'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
    'porcentagem_responsaveis_renda_ate_3_salarios',
    # # 'porcentagem_responsaveis_menos_8_anos_de_estudo',
    # # 'anos_medios_estudo_responsavel',
    'renda_media_responsavel',
    'numero_de_domicilios_particulares_permanentes',
    'numero_de_domicilios_improvisados',
    'numero_de_pessoas_residentes',
    'porcentagem_de_responsaveis_com_menos_30_anos',
    'numero_medio_pessoas'
]

In [3]:
df = pd.read_csv('resultados/modelo.csv.zip')
df.fillna(0, inplace=True)
# rmsp = (df.rm == 20)
subnormal = (df.tipo_do_setor == 1)
df.loc[:, 'subnormal'] = 0
df.loc[subnormal, 'subnormal'] = 1

/tmp/ipykernel_123746/3615071609.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resultados/modelo.csv.zip')


In [4]:
r_subnormal = df.loc[(subnormal), ['rm', 'subnormal']].groupby('rm').count().reset_index()
cidades_fora_rms = r_subnormal[(r_subnormal.subnormal < 20) | (r_subnormal.rm == 0.)].rm.to_list()
rms = r_subnormal[(r_subnormal.subnormal >= 20) & (r_subnormal.rm != 0.)].rm.to_list()

In [5]:
def processa_modelos(filter, rm=None, uf=None):

    registro = {}

    # filter = (df.rm == rm)
    df_rm = df.loc[(filter)]

    # print(df_rm.nome_rm.unique()[0])
    if rm:
        registro['nome_rm_ou_UF'] = df_rm.nome_rm.unique()[0]
    
    if uf:
        registro['nome_rm_ou_UF'] = uf
    # print(df_rm.subnormal.value_counts())

    setores_normais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 0)].shape[0]
    registro['quantidade_original_de_setores_normais'] = setores_normais
    
    setores_subnormais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 1)].shape[0]
    registro['quantidade_original_de_setores_subnormais'] = setores_subnormais
    # print(setores_normais, setores_subnormais)

    samples = min(setores_normais, setores_subnormais)

    # prepara amostras e lições
    x = df_rm.loc[(filter) & (subnormal)]
    x_ = df_rm.loc[(filter) & (~subnormal)].sample(n=samples, random_state=51)
    X = pd.concat([x, x_]).loc[(filter), dimensoes]
    y = pd.concat([x, x_]).loc[(filter), 'subnormal']

    # Processa os modelos, LDA
    # print(y.value_counts())
    clf = LinearDiscriminantAnalysis(solver='lsqr')
    clf.fit(X, y)
    # print(np.unique(clf.predict(X), return_counts=True))
    predicao = clf.predict(df_rm.loc[(filter), dimensoes])
    atribuidos_como_subnormais = np.unique(predicao, return_counts=True)[1][1]
    registro['atribuidos_como_subnormais_LDA'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_LDA'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_LDA'] = clf.score(X, y)
    df.loc[(filter), 'subnormal_LDA'] = predicao

    # Processa model RandonForest
    clf = RandomForestClassifier(max_depth=10, random_state=0)
    clf.fit(X, y)
    # print(np.unique(clf.predict(X), return_counts=True))
    predicao = clf.predict(df_rm.loc[(filter), dimensoes])
    atribuidos_como_subnormais = np.unique(predicao, return_counts=True)[1][1]
    registro['atribuidos_como_subnormais_RandomForest'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_RandomForest'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_RandomForest'] = clf.score(X, y)
    df.loc[(filter), 'subnormal_RandomForest'] = predicao


    # Processa modelo SVC
    clf = LinearSVC(random_state=0, tol=1e-5)
    clf.fit(Normalizer().fit_transform(X), y)
    # print(np.unique(clf.predict(X), return_counts=True))
    predicao = clf.predict(Normalizer().fit_transform(df_rm.loc[(filter), dimensoes]))
    atribuidos_como_subnormais = np.unique(clf.predict(Normalizer().fit_transform(df_rm.loc[(filter), dimensoes])), return_counts=True)[1][1]
    registro['atribuidos_como_subnormais_SVC'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_SVC'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_SVC'] = clf.score(Normalizer().fit_transform(X), y)
    df.loc[(filter), 'subnormal_SVC'] = predicao

    # Salva as predicoes em atributos
    return registro

In [6]:
# Processa cada um dos municípios da sua Região MEtropolitana
result = [] 

for rm in rms:

    rm_filter = (df.rm == rm)
    registro = processa_modelos(rm_filter, rm=rm)
    # Salva as predicoes em atributos
    result.append(registro)

In [7]:
df.subnormal_LDA.value_counts()

subnormal_LDA
0.0    83161
1.0    37553
Name: count, dtype: int64

In [8]:
df.columns

Index(['Cod_setor', 'uf', 'nome_UF', 'rm', 'nome_rm', 'meso_regiao',
       'cod_municipio', 'nome_municipio', 'situacao_setor', 'tipo_do_setor',
       'porcentagem_domicilios_sem_coleta_de_lixo',
       'porcentagem_domicilios_sem_ligacao_rede_de_agua',
       'porcentagem_domicilios_sem_banheiros',
       'porcentagem_domicilios_sem_esgoto_ou_fossa',
       'porcentagem_domicilios_cedidos_outra_forma',
       'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
       'numero_de_banheiros_por_habitante',
       'porcentagem_responsaveis_nao_alfabetizado',
       'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
       'porcentagem_responsaveis_renda_ate_3_salarios',
       'renda_media_responsavel',
       'numero_de_domicilios_particulares_permanentes',
       'numero_de_domicilios_improvisados', 'numero_de_pessoas_residentes',
       'porcentagem_de_responsaveis_com_menos_30_anos', 'numero_medio_pessoas',
       'subnormal', 'subnormal_LDA', 'subnormal_RandomFor

In [9]:
r_ufs = df[df.rm.isin(cidades_fora_rms)].loc[(subnormal), ['nome_UF', 'subnormal']].groupby('nome_UF').count().reset_index()

In [10]:
r_ufs

,nome_UF,subnormal
0,AC,33
1,AL,11
2,AM,94
3,AP,30
4,BA,39
5,CE,15
6,ES,112
7,GO,9
8,MG,94
9,MS,9


In [11]:
df_setores = df[df.rm.isin(cidades_fora_rms)].loc[:, ['nome_UF', 'subnormal', 'Cod_setor']].groupby('nome_UF').agg({'subnormal': 'sum', 'Cod_setor':'count'})

In [12]:
df_setores[df_setores.subnormal < 20]

,subnormal,Cod_setor
nome_UF,,
AL,11,2410
CE,15,8796
GO,9,7953
MA,0,6663
MS,9,4207
MT,0,4411
PB,0,2898
PI,0,3863
RN,5,2714


In [13]:
df_setores[df_setores.subnormal < 20].sum()

subnormal       57
Cod_setor    49014
dtype: int64

In [14]:
cidades_fora_rms

[0.0, 6.0, 12.0, 24.0, 28.0, 29.0, 36.0]

In [15]:
r_ufs[r_ufs.subnormal >= 20]

,nome_UF,subnormal
0,AC,33
2,AM,94
3,AP,30
4,BA,39
6,ES,112
8,MG,94
10,PA,127
11,PE,29
12,PR,53
13,RJ,571


In [16]:
r_ufs[r_ufs.subnormal < 20]

,nome_UF,subnormal
1,AL,11
5,CE,15
7,GO,9
9,MS,9
14,RN,5
16,RR,3
20,TO,5


In [17]:
df[(df.rm.isin(cidades_fora_rms))]

,Cod_setor,uf,nome_UF,rm,nome_rm,meso_regiao,cod_municipio,nome_municipio,situacao_setor,tipo_do_setor,...,renda_media_responsavel,numero_de_domicilios_particulares_permanentes,numero_de_domicilios_improvisados,numero_de_pessoas_residentes,porcentagem_de_responsaveis_com_menos_30_anos,numero_medio_pessoas,subnormal,subnormal_LDA,subnormal_RandomForest,subnormal_SVC
0,120001305000001,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,1.0,0.0,...,2134.060284,282,0.0,957.0,0.159574,3.393617,0,NaN,NaN,NaN
1,120001305000002,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,1.0,0.0,...,1120.444767,342,2.0,1203.0,0.244186,3.517544,0,NaN,NaN,NaN
2,120001305000003,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,1.0,0.0,...,1101.879917,482,1.0,1700.0,0.293996,3.526971,0,NaN,NaN,NaN
3,120001305000004,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,5.0,0.0,...,1110.433962,53,0.0,182.0,0.226415,3.433962,0,NaN,NaN,NaN
4,120001305000005,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,8.0,0.0,...,1176.864583,96,0.0,305.0,0.197917,3.177083,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310115,170210905000098,0,TO,0.0,0,0.0,0.0,0,0.0,0.0,...,1348.646240,359,0.0,0.0,0.278552,0.000000,0,NaN,NaN,NaN
310116,170210905000099,0,TO,0.0,0,0.0,0.0,0,0.0,0.0,...,963.225352,354,1.0,0.0,0.239437,0.000000,0,NaN,NaN,NaN
310117,170210905000100,0,TO,0.0,0,0.0,0.0,0,0.0,0.0,...,1307.174556,338,0.0,0.0,0.230769,0.000000,0,NaN,NaN,NaN
310118,170210905000101,0,TO,0.0,0,0.0,0.0,0,0.0,0.0,...,968.094340,265,0.0,0.0,0.362264,0.000000,0,NaN,NaN,NaN


In [18]:
# Processa cada uma das UFs
df_uf = df[df.rm.isin(cidades_fora_rms)]
ufs = r_ufs[r_ufs.subnormal >= 20].nome_UF.to_list()

for uf in ufs:

    uf_filter = (df.nome_UF == uf) & (df.rm.isin(cidades_fora_rms))
    print(uf)

    registro = processa_modelos(uf_filter, uf=uf)
    # Salva as predicoes em atributos
    result.append(registro)


AC
AM
AP
BA
ES
MG
PA
PE
PR
RJ
RO
RS
SC
SP2


In [22]:
pd.DataFrame(result).to_csv('resultados/geral_por_RM_UF.csv')

In [20]:
df.subnormal_LDA.value_counts()

subnormal_LDA
0.0    190277
1.0     65695
Name: count, dtype: int64

In [21]:
df.subnormal.value_counts()

subnormal
0    294305
1     15815
Name: count, dtype: int64

In [23]:
df.to_csv('resultados/modelo_preditivo.csv.zip', compression='zip')